In [11]:
import numpy as np
import os
import subprocess
import csv

In [12]:
# build work path from git root path
gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
WORK_DIR = os.getcwd()
print(WORK_DIR)

/home/adele/code/my28brains


In [13]:
# DATA_PATH = os.path.join(WORK_DIR, "data","28_HormoneData.txt")
DATA_PATH = os.path.join(WORK_DIR, "data","hormones.csv")

In [14]:
# data = np.loadtxt(DATA_PATH)
with open(DATA_PATH) as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)
print (d[0][2]) # 248

IndexError: list index out of range

In [15]:
d

[['dayID,Estro,Prog,Test,DHEAS,LH,FSH,SHBG,EthinylEstradiol,Levonorgestresl,CycleDay'],
 ['1,52.4,11.9,98.7,192.88,8.61,5.06,55.72,,,21'],
 ['2,74.2,10.6,81.2,247.12,4.89,4.13,58.22,,,22'],
 ['3,91.7,12.4,96.3,226.23,5.98,3.49,56.92,,,23'],
 ['4,94.6,13.5,90.5,212.84,4.32,3.65,57.29,,,24'],
 ['5,110,15,77.6,217.6,3.44,3.3,58.87,,,25'],
 ['6,117,15.5,85.2,254.84,2.58,3.08,63.26,,,26'],
 ['7,125,13.3,78.7,250.41,3.55,3.26,58.36,,,27'],
 ['8,123,8.78,88.9,271.71,5.21,4.47,61.78,,,28'],
 ['9,109,6.05,75.8,288.47,4.76,4.76,62.44,,,29'],
 ['10,73.8,2.46,87.3,204.74,3.32,7.94,53.76,,,30'],
 ['11,31.7,0.63,58.9,224.66,5.21,7.82,61.74,,,1'],
 ['12,22.9,0.28,70.4,233.79,5.29,7.85,54.77,,,2'],
 ['13,21.8,0.17,76.4,240.26,6.42,7.27,57.52,,,3'],
 ['14,36.7,0.084,61.6,244.45,5.04,7.12,53.98,,,4'],
 ['15,26.7,0.056,72.3,272.85,6.16,7.19,54.61,,,5'],
 ['16,31.4,0.073,74.5,237.81,6.08,7.09,56.71,,,6'],
 ['17,34.3,0.045,77.2,205.92,7.01,6.17,54.88,,,7'],
 ['18,51,0.037,95.2,262.48,6.82,5.35,55.83,,,8'],

In [16]:
import pandas as pd

In [17]:
df = pd.read_csv(DATA_PATH, delimiter=",")

In [18]:
df

,dayID,Estro,Prog,Test,DHEAS,LH,FSH,SHBG,EthinylEstradiol,Levonorgestresl,CycleDay
0,1,52.40,11.900,98.70,192.88,8.61,5.06,55.72,NaN,NaN,21
1,2,74.20,10.600,81.20,247.12,4.89,4.13,58.22,NaN,NaN,22
2,3,91.70,12.400,96.30,226.23,5.98,3.49,56.92,NaN,NaN,23
3,4,94.60,13.500,90.50,212.84,4.32,3.65,57.29,NaN,NaN,24
4,5,110.00,15.000,77.60,217.60,3.44,3.30,58.87,NaN,NaN,25
5,6,117.00,15.500,85.20,254.84,2.58,3.08,63.26,NaN,NaN,26
6,7,125.00,13.300,78.70,250.41,3.55,3.26,58.36,NaN,NaN,27
7,8,123.00,8.780,88.90,271.71,5.21,4.47,61.78,NaN,NaN,28
8,9,109.00,6.050,75.80,288.47,4.76,4.76,62.44,NaN,NaN,29
9,10,73.80,2.460,87.30,204.74,3.32,7.94,53.76,NaN,NaN,30


# Animations


In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

In [8]:
df[df["dayID"]< 31]

NameError: name 'df' is not defined

In [9]:

dayID=1
f"pic{dayID:02d}.png"

'pic01.png'

In [15]:
times = df[df["dayID"]< 21]["dayID"].values
times
df = df[df["dayID"]< 31]
print(df)

    dayID  Estro    Prog  Test   DHEAS     LH    FSH   SHBG EthinylEstradiol  \
0       1   52.4  11.900  98.7  192.88   8.61   5.06  55.72              NaN   
1       2   74.2  10.600  81.2  247.12   4.89   4.13  58.22              NaN   
2       3   91.7  12.400  96.3  226.23   5.98   3.49  56.92              NaN   
3       4   94.6  13.500  90.5  212.84   4.32   3.65  57.29              NaN   
4       5  110.0  15.000  77.6  217.60   3.44   3.30  58.87              NaN   
5       6  117.0  15.500  85.2  254.84   2.58   3.08  63.26              NaN   
6       7  125.0  13.300  78.7  250.41   3.55   3.26  58.36              NaN   
7       8  123.0   8.780  88.9  271.71   5.21   4.47  61.78              NaN   
8       9  109.0   6.050  75.8  288.47   4.76   4.76  62.44              NaN   
9      10   73.8   2.460  87.3  204.74   3.32   7.94  53.76              NaN   
10     11   31.7   0.630  58.9  224.66   5.21   7.82  61.74              NaN   
11     12   22.9   0.280  70.4  233.79  

In [1]:
def generate_plot(dayID):
    num_geodesic_frames = 5
    for frame in range(num_geodesic_frames):
        times = df[df["dayID"]< dayID]["dayID"]
        print(times)
        hormones_estro = df[df["dayID"]< dayID]["Estro"]
        hormones_progest = df[df["dayID"]< dayID]["Prog"]
        hormones_lh = df[df["dayID"]< dayID]["LH"]
        hormones_fsh = df[df["dayID"]< dayID]["FSH"]
        fig = plt.figure()
        ax = fig.add_subplot(111)
        estro = ax.plot(times, hormones_estro)
        progest = ax.plot(times, hormones_progest)
        lh = ax.plot(times, hormones_lh)
        fsh = ax.plot(times, hormones_fsh)
        # estro.set_label('Estrogen')
        # progest.set_label('Progesterone')
        # lh.set_label('LH')
        # fsh.set_label('FSH')
        if dayID > 2:
            estro = ax.scatter(times.values[-1], hormones_estro.values[-1], s=100)
            progest = ax.scatter(times.values[-1], hormones_progest.values[-1], s=100)
            lh = ax.scatter(times.values[-1], hormones_lh.values[-1], s=100)
            fsh = ax.scatter(times.values[-1], hormones_fsh.values[-1], s=100)
        ax.set_xlim((0, 30))
        ax.set_ylim(0,df["Estro"].max())
        ax.legend([estro,progest,lh,fsh],['Estrogen', 'Progesterone', 'LH', 'FSH'], loc='upper left')
        fig.savefig(f"{WORK_DIR}/data/hormones/dot_pic{dayID:02d}_frame{frame:01d}.svg")
        plt.close()

df = df[df["dayID"]< 31]
n_days = 30 #60 to include OC cycle
Parallel(n_jobs=-1)(delayed(generate_plot)(i) for i in range(1,n_days+1))

# os.system(f"ffmpeg -i {WORK_DIR}/data/hormones/dot_pic%02d_frame%01d.svg -vf palettegen palette.svg")
# os.system(f"ffmpeg -framerate 24 -i {WORK_DIR}/data/hormones/dot_pic%02d_frame%01d.svg -i palette.svg -lavfi paletteuse -framerate 24 {WORK_DIR}/data/hormones/dot_video.gif")



import glob
import os

ffmpeg -framerate 10 -pattern_type glob -i 'dot_pic*_frame*.png' -vf palettegen
palette.png && ffmpeg -framerate 10 -pattern_type glob -i 'dot_pic*_frame*.png' -i palette.png -filter_complex "paletteuse" output.gif

# palette_path = os.path.join(WORK_DIR, "data/hormones/palette.png")
# img_paths = sorted(glob.glob(os.path.join(WORK_DIR, "data/hormones/dot_pic*_frame*.png")))

# # Generate palette file
# palette_cmd = f"ffmpeg -i {' '.join(img_paths)} -vf palettegen {palette_path}"
# if os.system(palette_cmd) != 0:
#     print("Error generating palette file")
#     exit()

# # Generate GIF using palette file
# gif_cmd = f"ffmpeg -framerate 24 -i {' '.join(img_paths)} -i {palette_path} -lavfi paletteuse -framerate 24 {os.path.join(WORK_DIR, 'data/hormones/dot_video.gif')}"
# if os.system(gif_cmd) != 0:
#     print("Error generating GIF file")
#     exit()
# else:
    
#     print("GIF file generated successfully")

# img_paths = sorted(glob.glob(os.path.join(WORK_DIR, "data/hormones/dot_pic*_frame*.png")))
# palette_path = os.path.join(WORK_DIR, "data/hormones/palette.png")

# os.system(f"ffmpeg -i {' '.join(img_paths)} -vf palettegen {WORK_DIR}/data/hormones/palette.png")
# os.system(f"ffmpeg -framerate 24 -i {' '.join(img_paths)} -i {WORK_DIR}/data/hormones/palette.png -lavfi paletteuse -framerate 24 {os.path.join(WORK_DIR, 'data/hormones/dot_video.gif')}")


# os.system(f"ffmpeg -i {' '.join(img_paths)} -vf palettegen {' '.join(palette_path)}")
# os.system(f"ffmpeg -framerate 24 -i {' '.join(img_paths)} -i {' '.join(palette_path)} -lavfi paletteuse -framerate 24 {os.path.join(WORK_DIR, 'data/hormones/dot_video.gif')}")


SyntaxError: invalid syntax (1908268346.py, line 43)

In [20]:
import subprocess
import os
import glob

palette_path = os.path.join(WORK_DIR, "data/hormones/palette.png")
img_paths = sorted(glob.glob(os.path.join(WORK_DIR, "data/hormones/dot_pic*_frame*.png")))

# Generate palette file
palette_cmd = ["ffmpeg", "-i", " ".join(img_paths), "-vf", "palettegen", palette_path]
palette_result = subprocess.run(palette_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if palette_result.returncode != 0:
    print("Error generating palette file:")
    print(palette_result.stderr.decode())
    exit()

# Generate GIF using palette file
gif_cmd = ["ffmpeg", "-framerate", "24", "-i", " ".join(img_paths), "-i", palette_path, "-lavfi", "paletteuse", "-framerate", "24", os.path.join(WORK_DIR, "data/hormones/dot_video.gif")]
gif_result = subprocess.run(gif_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if gif_result.returncode != 0:
    print("Error generating GIF file:")
    print(gif_result.stderr.decode())
    exit()

else:
    print("GIF file generated successfully")

Error generating palette file:
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-l

In [21]:
os.getcwd()

'/home/adele/code/my28brains'

/tmp/ipykernel_4028211/1835829134.py:27: UserWarning: Legend does not support handles for list instances.
A proxy artist may be used instead.
See: https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#controlling-the-legend-entries
/tmp/ipykernel_4028211/1835829134.py:27: UserWarning: Legend does not support handles for list instances.
A proxy artist may be used instead.
See: https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#controlling-the-legend-entries


Series([], Name: dayID, dtype: int64)
Series([], Name: dayID, dtype: int64)
Series([], Name: dayID, dtype: int64)
Series([], Name: dayID, dtype: int64)
Series([], Name: dayID, dtype: int64)
0    1
Name: dayID, dtype: int64
0    1
Name: dayID, dtype: int64
0    1
Name: dayID, dtype: int64
0    1
Name: dayID, dtype: int64
0    1
Name: dayID, dtype: int64
0    1
1    2
2    3
3    4
Name: dayID, dtype: int64
0    1
1    2
2    3
3    4
Name: dayID, dtype: int64
0    1
1    2
2    3
3    4
Name: dayID, dtype: int64
0    1
1    2
2    3
3    4
Name: dayID, dtype: int64
0    1
1    2
2    3
3    4
Name: dayID, dtype: int64
0    1
1    2
2    3
3    4
4    5
Name: dayID, dtype: int64
0    1
1    2
2    3
3    4
4    5
Name: dayID, dtype: int64
0    1
1    2
2    3
3    4
4    5
Name: dayID, dtype: int64
0    1
1    2
2    3
3    4
4    5
Name: dayID, dtype: int64
0    1
1    2
2    3
3    4
4    5
Name: dayID, dtype: int64
0    1
1    2
2    3
3    4
4    5
5    6
6    7
7    8
Name: dayID, d

In [ ]:
ffmpeg --version